# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os


## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Transfer learning on staphylococcal transcriptomics:GLEAning uncharacterized genes	Talk	talk-5	Northeast and CT Valley ASM	4/25/25	"Worcester, MA"		Talk at Northeast and Connecticut Valley Joint Branch ASM Meeting
Transfer learning on staphylococcal transcriptomics: exploring uncharacterized genes	Talk	talk-1	Duke Microbiome Center	1/17/25	"Durham, NC"		Research in Progress Seminar (virtual)
Staphylococcal CRISPRi library for gene essentiality testing	Talk	talk-2	Yale University	11/15/25	"New Haven, CT"		Guest Seminar for the Yale Phage Center
"GLEAn: Graphs Learned from Expression for Annotation, a proof-of -oncept study in staphylococci"	Talk	talk-3	"Ipswitch, MA"	12/13/24	"London, UK"		Invited Seminar at New England Biolabs
Wrangling genomic and transcriptomic diversity of skin staphylococci	Talk	talk-4	Lake Arrowhead Microbial Genomics	9/17/24	"Lake Arrowhead, CA"		Talk at LAMG Meeting

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [48]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)


talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Transfer learning on staphylococcal transcript...,Talk,talk-5,Northeast and CT Valley ASM,2025-04-25,"Worcester, MA",NaN,Talk at Northeast and Connecticut Valley Joint...
1,Transfer learning on staphylococcal transcript...,Talk,talk-1,Duke Microbiome Center,2025-01-17,"Durham, NC",NaN,Research in Progress Seminar (virtual)
2,Staphylococcal CRISPRi library for gene essent...,Talk,talk-2,Yale University,2025-11-15,"New Haven, CT",NaN,Guest Seminar for the Yale Phage Center
3,GLEAn: Graphs Learned from Expression for Anno...,Talk,talk-3,New England Biolabs,2024-12-13,"Ipswitch, MA",NaN,Invited Seminar at New England Biolabs
4,Wrangling genomic and transcriptomic diversity...,Talk,talk-4,Lake Arrowhead Microbial Genomics,2024-09-17,"Lake Arrowhead, CA",NaN,Talk at LAMG Meeting
5,Decoding microbial diversity on the skin,Talk,talk-7,CT State Northwestern,2024-06-07,"Winstead, CT",NaN,Keynote at the New England Biology Association...
6,Transfer learning on staphylococcal transcript...,Talk,talk-6,DartCF Retreat,2024-05-04,"Hanover, NH",NaN,Talk at the DartCF Reatreat at Dartmouth
7,Transfer learning on staphylococcal transcript...,Talk,talk-8,The Jackson Laboratory for Genomic Medicine,2024-03-01,"Farmington, CT",NaN,Trainee Talk at JAX-GM
8,Wrangling Microbial Data,Talk,talk-9,Union College,2023-10-05,"Schenectady, NY",NaN,"Seminar hosted by the Math, Computer Science a..."
9,Wrangling Microbial Diversity,Talk,talk-10,Bard College,2022-11-10,"Annandale on Hudson, NY",NaN,Bio Seminar at Bard College


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [49]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [50]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [51]:
!ls ../_talks

2012-03-01-talk-1.md     2020-01-08-talk-18.md    2023-10-05-talk-9.md
2013-03-01-tutorial-1.md 2020-09-10-talk-17.md    2024-03-01-talk-8.md
2014-02-01-talk-2.md     2020-12-17-talk-16.md    2024-05-04-talk-6.md
2014-03-01-talk-3.md     2021-03-22-talk-15.md    2024-06-07-talk-7.md
2015-05-21-talk-23.md    2021-11-19-talk-14.md    2024-09-17-talk-4.md
2017-06-08-talk-22.md    2022-06-01-talk-13.md    2024-12-13-talk-3.md
2018-01-22-talk-20.md    2022-06-09-talk-12.md    2025-01-17-talk-1.md
2019-04-18-talk-21.md    2022-11-02-talk-11.md    2025-04-25-talk-5.md
2019-06-28-talk-19.md    2022-11-10-talk-10.md    2025-11-15-talk-2.md


In [54]:
!cat ../_talks/2020-12-17-talk-16.md


---
title: "Well-spoken killer whales: should we believe what we hear?"
collection: talks
type: "Talk"
permalink: /talks/2020-12-17-talk-16
venue: "Dartmouth College"
date: 2020-12-17
location: "Hanover, NH"
---

[More information here](https://youtu.be/MEcZPuQYhn4?si=m3ILNBISfdrnNag6&t=1037)

Science on Tap Talk for the Community (virtual)
